In [26]:
import json
import os
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations

from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator

import plotly.graph_objects as go
from plotly.subplots import make_subplots
test_projects = [
    "HomeA/2016",
    "HomeB/2016",
    "HomeC/2016",
    # "HomeD/2016",
    "HomeF/2016",
    "HomeG/2016",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 20,
            },
            # "humidity#NUM" : {
            #     "range" : (0., 1.0),
            #     "interval" : 0.1,
            # },
            # "apparentTemperature#NUM" : {
            #     "range" : (-10., 100),
            #     "interval" : 10,
            # },
            # "summary#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
        })
capacity = {
    "Range" : 1,
    "Microwave": 1,
    "LivingLights": 0,
    "HomeOffice": 1,
    "WashingMachine": 1,
    # "Window": 0,
}
BOOL_SIM = False
test_dates = generate_test_date(test_projects, test_ratio = 0.3, true_random=False, is_sim=BOOL_SIM)


In [27]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], grid_data = test_umass(test_project=p, ctx_info=ctx_info, ccp_alpha=1e-4, test_dates=test_dates, is_sim=BOOL_SIM)

In [28]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [29]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # for g in d_groups:
        #     print(ctx_info.translate_ctx_box(g["box"][0] + g["box"][1]))
        #     print(g["dis"])


The habit groups found in HomeA/2016
Range 3
Microwave 1
LivingLights 33
HomeOffice 12
The habit groups found in HomeB/2016
LivingLights 9
Microwave 1
The habit groups found in HomeC/2016
HomeOffice 9
Microwave 1
LivingLights 31
The habit groups found in HomeF/2016
Microwave 1
WashingMachine 4
The habit groups found in HomeG/2016
Range 4
LivingLights 5
HomeOffice 11
Microwave 3


In [30]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

{'Range': 6, 'Microwave': 18, 'LivingLights': 0, 'HomeOffice': 79, 'WashingMachine': 0}


In [31]:
import plotly.express as px
# for x in final_conflicts["Window"]:
#     print((ctx_info.translate_ctx_box(x["box"]),x["prob"]))
# probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["Window"])]
# fig = px.histogram(probs)
# fig.show()

In [32]:
import copy
# Make ground truth:
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(p, is_sim=BOOL_SIM)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": ctx_info,
    "capacity": capacity
}
# test_dates = generate_test_date(test_projects, test_ratio = 1., true_random=False)

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

191


In [33]:

# TEST_RATIO = 0.3
# conflict_time = [x['cur_time'] for x in conflicts]
# end_time = max(conflict_time)
# start_time = min(conflict_time)
# print(start_time)
# print(end_time)
# total_time_range = end_time - start_time
# test_start = end_time - total_time_range * TEST_RATIO
# print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 10

for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    if test_state_cnt[s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})


{'Range': 2, 'Microwave': 19, 'LivingLights': 0, 'HomeOffice': 170, 'WashingMachine': 0}
{'Range': 2, 'Microwave': 14, 'LivingLights': 0, 'HomeOffice': 75, 'WashingMachine': 0}


In [34]:
it = np.nditer(test_state_cnt, flags=['multi_index'])
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_error = 0
exp_gt_c = 0
exp_result = {d:[0,0,0,0, 0] for d in all_devices}
max_p = 0
for count in it:
    if count < MIN_TEST_OBS:
        continue
    state = it.multi_index
    for d in all_devices:
        for u_pair in user_pairs:
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (frozenset(u_pair) in conflict_state_device[d][state]):
                gt_prob =  float(conflict_state_device[d][state][frozenset(u_pair)]) / count
            pred_prob = conflict_predicator.get_prob_conflict(state, u_pair, d)
            exp_cnt += 1
            if gt_prob > 0:
                # if pred_prob <= 0.001:
                #     print(ctx_info.translate_ctx_point(state).values(), gt_prob, pred_prob)
                exp_error += abs(pred_prob - gt_prob)
                exp_gt_c += 1
                exp_result[d][0] += abs(pred_prob - gt_prob)
                exp_result[d][1] += 1
                exp_result[d][4] += gt_prob
                max_p = max(gt_prob, max_p)
            else:
                exp_result[d][2] += abs(pred_prob - gt_prob)
                exp_result[d][3] += 1

for d in exp_result:
    if exp_result[d][1] > 0:
        exp_result[d][0] = exp_result[d][0] / exp_result[d][1]
        exp_result[d][4] = exp_result[d][4] / exp_result[d][1]
    if exp_result[d][3] > 0:
        exp_result[d][2] = exp_result[d][2] / exp_result[d][3]

print(exp_result)

{'Range': [0.023766693210107524, 2, 6.053456936148705e-05, 5038, 0.03333333333333333], 'Microwave': [0.04125416232764394, 14, 0.00017984335212940084, 5026, 0.041470872179076516], 'LivingLights': [0, 0, 0.0, 5040, 0], 'HomeOffice': [0.03497984687777625, 90, 0.001326184699759625, 4950, 0.06071370248707792], 'WashingMachine': [0, 0, 0.0, 5040, 0]}


In [35]:
missed_gt_conflicts = {d:[] for d in conflict_device}
hit_gt_conflicts = {d:{} for d in conflict_device}
for d in final_conflicts:
    hit_gt_conflicts[d] = {i:0 for i in range(len(final_conflicts[d]))}

prob_loss = {}
sum_pred = 0
sum_gt = 0
err = 0
cntt = 0
for d, conflicts_d in conflict_state_device.items():
    for s, conflicts in conflicts_d.items():
        for users, cnt in conflicts.items():
            flag = False
            for idx, c_predict in enumerate(final_conflicts[d]):
                if not match_user_groups([x[0] for x in c_predict["users"]], users):
                    continue
                if does_contain_point(c_predict["box"], s):
                    pred_prob = c_predict["prob"]
                    gt_prob = float(cnt) / test_state_cnt[s]
                    sum_pred += pred_prob
                    sum_gt += gt_prob
                    err += abs(gt_prob - pred_prob)
                    cntt += 1
                    flag = True
            if not flag:
                print("????")
print(sum_gt/cntt, sum_pred/cntt, err/cntt)



0.05765558585859196 0.02525230846885295 0.03559696111327445


In [36]:
# def match_pair(predicted_users, gt_users):
#     return set(predicted_users) == set(gt_users)
# missed_gt_conflicts = {d:[] for d in conflict_device}
# hit_gt_conflicts = {d:{} for d in conflict_device}
# for d in final_conflicts:
#     hit_gt_conflicts[d] = {i:0 for i in range(len(final_conflicts[d]))}
# for d, conflicts in conflict_device.items():
#     for c in conflicts:
#         coor = ctx_info.get_coor_by_ctx(c["ctx"])
#         flag = False
#         for idx,c_predict in enumerate(final_conflicts[d]):
#             if not match_user_groups([x[0] for x in c_predict["users"]], c["device_states"].keys()):
#                 # print(c_predict["users"], c["device_states"])
#                 continue
#             if does_contain_point(c_predict["box"], coor):
#                 flag = True
#                 hit_gt_conflicts[d][idx] += 1
#                 break
#         if not flag:
#             missed_gt_conflicts[d].append(c)
# print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
# hit_count = {d:[0,0] for d in hit_gt_conflicts}
# for d,hit_c in hit_gt_conflicts.items():
#     for i,h in hit_c.items():
#         if h == 0:
#             hit_count[d][0] += 1
#         else:
#             hit_count[d][1] += 1
# # print(hit_gt_conflicts["Microwave"])
# # print(final_conflicts["Microwave"][14])
# print(hit_count)

In [37]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

draw_device = "HomeOffice"
c_point = [[],[]]
missed_point = [[], []]
for c in conflict_device[draw_device]:
    coor = ctx_info.get_coor_by_ctx(c["ctx"])
    c_point[0].append(coor[0])
    c_point[1].append(coor[1])
for c in missed_gt_conflicts[draw_device]:
    coor = ctx_info.get_coor_by_ctx(c["ctx"])
    missed_point[0].append(coor[0])
    missed_point[1].append(coor[1])
fig = go.Figure(data=go.Scatter(x=c_point[0], y=c_point[1], mode="markers"))
fig.add_trace(go.Scatter(x=missed_point[0], y=missed_point[1], mode="markers", marker=dict(color="red")))

for c in final_conflicts[draw_device]:
    fig.add_shape(type="rect",
        x0=c["box"][0], y0=c["box"][1], x1=c["box"][2], y1=c["box"][3],
        line=dict(color="RoyalBlue"),
    )
fig.show()


In [14]:
hit_miss_rate = [[],[]]
for prob_threshold in np.arange(0.0, 0.5, 0.001):
    test_conflict = {
        d: [x for x in final_conflicts[d] if x["prob"] >= prob_threshold]
        for d in final_conflicts
    }
    missed_gt_conflicts = {d:[] for d in conflict_device}
    hit_gt_conflicts = {d:{} for d in conflict_device}
    for d in test_conflict:
        hit_gt_conflicts[d] = {i:0 for i in range(len(test_conflict[d]))}
    for d, conflicts in conflict_device.items():
        for c in conflicts:
            coor = ctx_info.get_coor_by_ctx(c["ctx"])
            flag = False
            for idx,c_predict in enumerate(test_conflict[d]):
                if does_contain_point(c_predict["box"], coor):
                    flag = True
                    hit_gt_conflicts[d][idx] += 1
                    break
            if not flag:
                missed_gt_conflicts[d].append(c)
    all_missed_gt_conflicts = sum([len(x) for d,x in missed_gt_conflicts.items()])
    all_gt_conflicts = sum([len(x) for d,x in conflict_device.items()])
    hit_miss_rate[0].append(1.0 - float(all_missed_gt_conflicts) / all_gt_conflicts)
    # print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
    all_pred_conflicts = 0
    all_missed_pred_conflicts = 0
    hit_count = {d:[0,0] for d in hit_gt_conflicts}
    for d,hit_c in hit_gt_conflicts.items():
        for i,h in hit_c.items():
            all_pred_conflicts += 1
            if h == 0:
                all_missed_pred_conflicts += 1
                hit_count[d][0] += 1
            else:
                hit_count[d][1] += 1
    if all_pred_conflicts == 0:
        break
    hit_miss_rate[1].append(1.0 - float(all_missed_pred_conflicts) / all_pred_conflicts)
    # print(hit_count)


{'Range': {}, 'Microwave': {}, 'LivingLights': {}, 'HomeOffice': {}, 'WashingMachine': {}}


In [15]:
print(len(hit_miss_rate[0]))
from alpha_test import compute_auc
fig = go.Figure(data=go.Scatter(x=hit_miss_rate[0], y=hit_miss_rate[1], mode="lines+markers"))
fig.show()

print(compute_auc(hit_miss_rate[0], hit_miss_rate[1]))



77


IndexError: list index out of range

In [13]:
from rtree import index


draw = {
    "HomeA/2016": ["Range"],
    "HomeG/2016": ["Range"],
}
fig = make_subplots(rows=sum([len(draw[x]) for x in draw]), cols=1)
p = index.Property()
p.dimension = len(ctx_info.get_all_ctx_ordered())
row_id = 1
for home in draw:
    for d in draw[home]:
        r_tree = index.Index(properties=p)
        for i,x in enumerate(habit_groups[home][d]):
            bound = x["box"][0] + x["box"][1] 
            r_tree.insert(id=i, coordinates=bound, obj=sum(x["dis"]))
        on_rate_group = np.full(ctx_info.get_ctx_space_shape(), -0.1)

        for i in range(ctx_info.get_ctx_space_shape()[0]):
            for j in range(ctx_info.get_ctx_space_shape()[1]):
                result = list(r_tree.intersection([i,j,i,j], objects=True))

                on_rate_group[i,j] = result[0].object
        fig.append_trace(
            go.Heatmap(z=np.transpose(on_rate_group)),
            row_id, 1
        )
        row_id += 1
fig.show()

IndexError: list index out of range

In [ ]:
from ConflictDetector import device_capacity_conflict
print(ctx_info.get_space_area())
print(ctx_info.get_ctx_space_shape())
mins = ctx_info.get_coor_by_ctx({TIME_CTX:0, WEEKDAY_CTX:0})
maxs = ctx_info.get_coor_by_ctx({TIME_CTX:24*60, WEEKDAY_CTX:6})
print(compute_area([0,0, 72,6]))